In [42]:
import requests     # library untuk URL
from bs4 import BeautifulSoup   # library untuk scrapping web
import re
import pandas as pd
from lxml import html
import time
from csv import writer    # library untuk membuat file CSV
import os
from tqdm import tqdm

## Getting link

In [8]:
web = ["https://sidata-ptn.ltmpt.ac.id/ptn_sb.php","https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=-2"]
for i in range(len(web)):
    url = web[i]
    print(url)

https://sidata-ptn.ltmpt.ac.id/ptn_sb.php
https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=-2


In [39]:
#membuat folder baru untuk menyimpan file nanti
os.makedirs("main_panel")

In [45]:
#set link untuk scraping
#terdapat dua link yaitu link PTN dan link Politeknik
#setiap link berisi list universitas
#pertama dapatkan setiap link universitas tersebut
list_df = []
web = ["https://sidata-ptn.ltmpt.ac.id/ptn_sb.php","https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=-2"]
ptn_politeknik = ["ptn","politeknik"]
for i in tqdm(range(len(web))):
    url =  web[i] # inisialisasi alamat URL
    res = requests.get(url)     # menggunakan library request dengan mengirimkan header
    data = res.content
    soup = BeautifulSoup(data)
    #soup = BeautifulSoup(res.content, "html.parser")    # parsing html menggunakan Beautifulsoup (alternatif)
    rows = soup.find("table", {"class":"table table-striped"})     # menggunakan fungsi find untuk mencari class dari daftar informasi yang diinginkan
    #divs1 = rows.findAll("tr", {"valign" :"top"})   # mencari semua class yang ada pada variabel rows
    ling = rows.findAll("a", {"class":"btn btn-info btn-xs"}) 
    
    print(url)
    #mendapatkan kode link
    urls = ''
    for teks in ling:
        tek = teks.get("href")
        urls += tek
        urls += ','
    
#     print(urls)
    
    #memisahkan setiap kode berdasarkan koma
    urls = urls.split(',')
    
    #menggabungkan link utama dengan kode link
    links=[]
    for x in urls:
        sites = web[0] + x
        links.append(sites)
    
    #link terakhir hanya berisi link awal tanpa kode
    links = links[:-1]
    
#     print(links)
    
    #scrape data setiap links
#     list_df = []
    for y in range(len(links)):
        # Site URL
        url= links[y]

        # membuat get request
        html_content = requests.get(url).text

        # Parse HTML code
        soup = BeautifulSoup(html_content, "lxml")

        # mencari semua tabel yang akan discrap
        tabs = soup.find_all("table", attrs={"class":"table table-striped vtop"})

#         print("jumlah tabel adalah ", len(tabs))
        # pada class ini terdapat dua table yaitu saintek dan soshum
        # ambil tabel pertama yaitu index (0) untuk saintek lalu ubah menjadi (1) untuk soshum
        # namun tidak semua universitas memiliki fakultas di soshum maupun saintek
        # kemungkinan terdapat dataset yang indek pertamanya bukan soshum melainkan saintek sehingga perlu diubah nanti
        for e in range(len(tabs)):
            try:
                table1 = tabs[e] #ganti index sesuai jumlah table yang tersedia
                
                # menemukan semua table rows
                body = table1.find_all("tr")
                # nilai teratas dari table rows merupakan kolom tabel
                head = body[0] # set nama kolom dengan set index body (0)
                body_rows = body[1:] # sisa table rows merupakan isi maka set index body (1:) 

                # iterasi setiap data

                # looping nama kolom
                headings = []
                for item in head.find_all("th"): # nama kolom berada dalam elemen "th"
                    # convert elemen "th" berformat text dan strip "\n"
                    item = (item.text).rstrip("\n")
                    headings.append(item)

                # looping body_rows data

                all_rows = [] #berisi seluruh baris data
                for row_num in range(len(body_rows)):
                    row = [] #berisi data per satu baris
                    for row_item in body_rows[row_num].find_all("td"): 
                        # row_item.text menghapus tags
                        # regex menghapus \xa0, \n dan koma dari row_item.text
                        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
                        row.append(aa)
                    all_rows.append(row)

                #menggabungkan kolom dengan body
                df = pd.DataFrame(data=all_rows,columns=headings)
                df['KODE_PT'] = links[y][-3:] # ambil tiga angka akhir dari link (https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=911)
                ranah = ["saintek","soshum"]
                df['KATEGORI'] = ranah[e] # saintek/soshum
                df['TIPE'] = ptn_politeknik[i] #menambah kolom tipe (ptn/politeknik)
                df_name = links[y][-7:] +"("+ ranah[e] + ")" + ptn_politeknik[i] +".csv" #membuat nama file csv
                df.to_csv("main_panel/" + df_name) # ambil tujuh angka akhir dari link (https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=911)
                list_df.append(df_name) # membuat list berisi seluruh file csv yang dibuat
            except:
                continue #jika tabel tidak ada maka lanjutkan ke iterasi selanjutnya

        

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

https://sidata-ptn.ltmpt.ac.id/ptn_sb.php


 50%|██████████████████████████████████████████                                          | 1/2 [00:13<00:13, 13.40s/it]

https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=-2


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.71s/it]


In [27]:
print(list_df)

['ptn=111(saintek)ptn.csv', 'ptn=111(soshum)ptn.csv', 'ptn=112(saintek)ptn.csv', 'ptn=112(soshum)ptn.csv', 'ptn=113(saintek)ptn.csv', 'ptn=113(soshum)ptn.csv', 'ptn=114(saintek)ptn.csv', 'ptn=114(soshum)ptn.csv', 'ptn=115(saintek)ptn.csv', 'ptn=116(saintek)ptn.csv', 'ptn=116(soshum)ptn.csv', 'ptn=121(saintek)ptn.csv', 'ptn=121(soshum)ptn.csv', 'ptn=122(saintek)ptn.csv', 'ptn=122(soshum)ptn.csv', 'ptn=123(saintek)ptn.csv', 'ptn=123(soshum)ptn.csv', 'ptn=131(saintek)ptn.csv', 'ptn=131(soshum)ptn.csv', 'ptn=132(saintek)ptn.csv', 'ptn=132(soshum)ptn.csv', 'ptn=133(saintek)ptn.csv', 'ptn=133(soshum)ptn.csv', 'ptn=141(saintek)ptn.csv', 'ptn=141(soshum)ptn.csv', 'ptn=142(saintek)ptn.csv', 'ptn=142(soshum)ptn.csv', 'ptn=143(saintek)ptn.csv', 'ptn=151(saintek)ptn.csv', 'ptn=151(soshum)ptn.csv', 'ptn=161(saintek)ptn.csv', 'ptn=161(soshum)ptn.csv', 'ptn=171(saintek)ptn.csv', 'ptn=171(soshum)ptn.csv', 'ptn=172(saintek)ptn.csv', 'ptn=172(soshum)ptn.csv', 'ptn=181(saintek)ptn.csv', 'ptn=181(soshum)p

In [28]:
#test df politeknik
data = "ptn=221(soshum)politeknik.csv"
ptn221 = pd.read_csv(data)
ptn221.tail()

,Unnamed: 0,NO,KODE,NAMA,JENJANG,DAYA TAMPUNG 2021,PEMINAT 2020,JENIS PORTOFOLIO,KODE_PT,KATEGORI,TIPE
0,0,1,2212011,AKUNTANSI LEMBAGA KEUANGAN SYARIAH,D4,27,0,Tidak Ada,221,soshum,politeknik


In [29]:
# test df ptn
data = "ptn=751(soshum)ptn.csv"
ptn751 = pd.read_csv(data)
ptn751.tail()

,Unnamed: 0,NO,KODE,NAMA,JENJANG,DAYA TAMPUNG 2021,PEMINAT 2020,JENIS PORTOFOLIO,KODE_PT,KATEGORI,TIPE
26,26,27,7512285,PSIKOLOGI,S1,31,182.0,Tidak Ada,751,soshum,ptn
27,27,28,7512293,ILMU POLITIK,S1,73,74.0,Tidak Ada,751,soshum,ptn
28,28,29,7512304,PENDIDIKAN AKUNTANSI,S1,31,105.0,Tidak Ada,751,soshum,ptn
29,29,30,7512327,PERPUSTAKAAN DAN ILMU INFORMASI,S1,38,46.0,Tidak Ada,751,soshum,ptn
30,30,31,7512335,ILMU PEMERINTAHAN,S1,81,110.0,Tidak Ada,751,soshum,ptn


### Merge All
<p>
    Menggabungkan semua dataframe yang telah dibuat menjadi satu file
    
    

In [30]:
type(list_df)

list

In [31]:
len(list_df)

221

In [34]:
df_list = []
for i in range(len(list_df)):
    df_list.append(pd.read_csv(list_df[i]))
# print(df_list)
df = pd.concat(df_list)

In [35]:
df

,Unnamed: 0,NO,KODE,NAMA,JENJANG,DAYA TAMPUNG 2021,PEMINAT 2020,JENIS PORTOFOLIO,KODE_PT,KATEGORI,TIPE
0,0,1,1111014,PENDIDIKAN DOKTER HEWAN,S1,94,455.0,Tidak Ada,111,saintek,ptn
1,1,2,1111022,TEKNIK SIPIL,S1,77,741.0,Tidak Ada,111,saintek,ptn
2,2,3,1111037,TEKNIK MESIN,S1,66,338.0,Tidak Ada,111,saintek,ptn
3,3,4,1111045,TEKNIK KIMIA,S1,62,201.0,Tidak Ada,111,saintek,ptn
4,4,5,1111053,ARSITEKTUR,S1,61,410.0,Tidak Ada,111,saintek,ptn
...,...,...,...,...,...,...,...,...,...,...,...
2,2,3,8321034,MANAJEMEN REKAYASA PERIKANAN TANGKAP,D4,60,0.0,Tidak Ada,832,saintek,politeknik
3,3,4,8321042,REKAYASA BUDIDAYA LAUT,D4,60,0.0,Tidak Ada,832,saintek,politeknik
4,4,5,8321057,MANAJEMEN REKAYASA PENGOLAHAN HASIL PERIKANAN,D4,60,0.0,Tidak Ada,832,saintek,politeknik
0,0,1,8322014,AGROWISATA BAHARI,D4,60,0.0,Tidak Ada,832,soshum,politeknik


In [37]:
# save menjadi csv file
df.to_csv("full_ptn_politeknik.csv")